# Basic setup

First of all, we need to somehow have control of the class $\beta$ in WDVV. This is a class $\beta=\sum \iota_j \mathbf{L}_j$. We associate to this class a monomial $\prod z_j^{\iota_j}$. Conversely, from such a monomial, we can get a partition. Note that, due to symmetry, the invariants depend only on the partition, e.g. the invariants associated to $\mathbf{L}_1+\mathbf{L}_2$ are the same as the ones for $\mathbf{L}_3+\mathbf{L}_4$.

So, we first define variables z, as well as variables corresponing to the divisors $D=\sum d_\alpha \mathbb{E}_\alpha, D'=\sum dd_\alpha \mathbb{E}_\alpha$.

We also define a variable that we want to solve for corresponding to the one point and three point invariants $\langle \delta_i\rangle_{part}, \langle \delta_1, \delta_n, \delta\rangle_{part}$. Let's name the 1pt ones a and the 3pt ones b. We have that $a_1=1, \langle \delta_1, \delta_1, \delta_{k-2}\rangle_0=\frac{1}{k}$.

In [ ]:
# define the order to which we want to compute the GW invariants. In this example, if we want to compute everything up to degree 6, we set n=6. The total computation takes about 40 seconds. But for higher n, it slows down a bit.

n=6

#define variables and parameters
znames      = ["z%d"%i for i in range(1,n+1)]
dnames      = ["d{}".format(i) for i in range(1, n+1)]       # d1,...,dn
ddnames     = ["dd{}".format(i) for i in range(1, n+1)]      # dp1,...,dpn

def b_name(int,part):
    if part == ():
        return "b_trivial"
    else:
        return "b_{}_{}".format(int,"".join(map(str,part)))

b_paramnames = []
for btotal in range(1,n+1):
    for part in Partitions(btotal):
        for int in range(1,n+1):
            b_paramnames.append(b_name(int,tuple(part)))

def a_name(part):
    if part == ():
        return "a_trivial"
    else:
        return "a_{}".format("".join(map(str,part)))
a_paramnames = []
for atotal in range(1,n+1):
    for part in Partitions(atotal):
        a_paramnames.append(a_name(tuple(part)))

# define the ring
R=PolynomialRing(QQ,znames+dnames+ddnames+a_paramnames+b_paramnames)
gens = R.gens()
z=gens[0:n]
d=gens[n:2*n]
dd=gens[2*n:3*n]

We add a new variable k which is really the type of orbifold point.

In [80]:
nameswk = ['k']+list(R.variable_names())
S=PolynomialRing(R.base_ring(), nameswk)
S
k=S.gens()[0:1]

We define a function which takes a monomial in $z_i$ and produces a partition.

In [81]:
# associate a partition to a monomial. need to order them in decreasing order to define a partition
def partition_from_poly(poly):
    list = []
    alldegrees = poly.degrees()
    for i in alldegrees:
        if i > 0:
            list.append(i)
    return sorted(list, reverse=true)

Now, given a monomial, i want to produce the WDVV equations for that specific partition.

# Defining the RHS and LHS in WDVV

Now let's do WDVV for $\delta_1, \delta{n-1},\phi_D, \phi_{D'}$ in a class of degree $n/k$.

In [82]:
def kRHS_from_monomial(poly):
    kRHS=0
    # this is computing Ljcdot the class we are in+1, multiplied by k so that we are not in the fraction field
    A=poly.degree()-1+k[0]

    #this is encoding the common part of D' cdot the class. we need to also remove Dcdot L_j
    sum = 0
    for i in range(n):
        sum += R(poly).degree(z[i])*dd[i]


    for j in range(n):
        if R.monomial_divides(z[j],poly):
            ainv = a_name(partition_from_poly(R.monomial_quotient(poly,z[j])))
            kRHS+=d[j]*(S(A)-k[0]*poly.degree(z[j]))*(sum-dd[j])*S(ainv)
    return kRHS

Some examples:

In [83]:
#examples
print(kRHS_from_monomial(z[0]*z[1]*z[2]))
print(kRHS_from_monomial(z[0]^2))

2*d2*dd1*a_11 + 2*d3*dd1*a_11 + 2*d1*dd2*a_11 + 2*d3*dd2*a_11 + 2*d1*dd3*a_11 + 2*d2*dd3*a_11
-k*d1*dd1*a_1 + d1*dd1*a_1


Before defining the LHS of WDVV, we need to define the intersection of $D$ with $\beta$ where $\beta$ is associated to a monomial in $z_i$. The result is some linear function in the $d$ variables.

In [84]:
def divisor_intersection_d(poly): 
    divisorsum=0
    for i in range(n):
        divisorsum+=d[i]*poly.degree(z[i])
    return divisorsum

def divisor_intersection_dd(poly): 
    ddivisorsum=0
    for i in range(n):
        ddivisorsum+=dd[i]*poly.degree(z[i])
    return ddivisorsum

Here is an example:

In [85]:
divisor_intersection_dd(z[0]*z[3])

dd1 + dd4

Now we can define the LHS:

In [86]:
def LHS_from_monomial(poly):
    degree = poly.degree()
    deltadeg = degree-1
    #start with the degree 0 contribution for <delta1,deltan-1,delta>_0
    LHS=S(a_name(partition_from_poly(poly)))*divisor_intersection_d(poly)*divisor_intersection_dd(poly)

    
    for submonomial in R.monomial_all_divisors(poly):
        if submonomial != poly:
            partb = partition_from_poly(R(submonomial))
            parta = partition_from_poly(R.monomial_quotient(R(poly), R(submonomial)))
            quot = R.monomial_quotient(R(poly), R(submonomial))

            LHS+=k[0]*S(b_name(deltadeg,partb))*S(a_name(parta))*divisor_intersection_d(quot)*divisor_intersection_dd(quot)

    return LHS


We can extract the coefficients of this as a function of d,dd  which gives a function of the a and b unknonw GW invariants:

In [87]:
LHS_from_monomial(z[0]*z[1]*z[2]).coefficient({d[0]: 1, dd[0]:1})

2*k*a_11*b_2_1 + k*a_1*b_2_11 + a_111

# Solving by comparing coefficients of d,dd

Now, from a given class $\beta$ to which we associate a monomial, we can get some equations by comparing the d,dd contributions on the LHS and RHS.

In [88]:
def eqns_from_monomial(poly):
    eqns=[]
    kLHS =k[0]*LHS_from_monomial(poly)
    kRHS = kRHS_from_monomial(poly)
    for i in range(n):
        for j in range(n):
            if (kLHS-kRHS).coefficient({d[i]: 1, dd[j]:1}) !=0:
                eqns.append((kLHS-kRHS).coefficient({d[i]: 1, dd[j]:1})      )
    return eqns


Here is, for example, the relations which need to vanish when $\beta= \mathbf{L}+\mathbf{L}', \mathbf{L}\neq \mathbf{L}'$.

In [89]:
from IPython.display import display, Math
display(Math(latex(eqns_from_monomial(z[0]*z[1]))))

<IPython.core.display.Math object>

We have an initial condition, namely $\langle \delta_1 \rangle_{\mathbf{L}}=1$.

In [90]:
trivialeqn=[]
for part in Partitions(1):
    trivialeqn.append(S(a_name(Partitions(1)[0]))-1)

trivialeqn


[a_1 - 1]

For example, by hand we can then infer that $a_{1,1}=\langle \delta_2\rangle_{[1,1]}=\frac{1}{k}, b_{1,[1]}=\langle \delta_1, \delta_1, \delta_{k-1}\rangle_{[1]}=-\frac{1}{k^2}$.

Now, we create a representative set of monomials for each partition of a given degree. Then, we introduce all the equations for all representative classes.

In [91]:
#degree d representative polynomials
def degdpolys(d):
    all=[]
    for partition in Partitions(d):
        mon=1
        for l in range(len(partition)):
            mon*=z[l]^partition[l]
        all.append(mon)
    return all

#eqns in degree m
def eqns(m):
    total=[]
    for p in degdpolys(m):
        total+=eqns_from_monomial(p)
    return total

For example: the representative polynomials of degree 4 are associated to the partitions $[4],[3,1], [2,1,1], [1,1,1,1]$.

In [92]:
print(degdpolys(4))
print(eqns(3))


[z1^4, z1^3*z2, z1^2*z2^2, z1^2*z2*z3, z1*z2*z3*z4]
[4*k^2*a_2*b_2_1 + k^2*a_1*b_2_2 + 4*k*a_2 + 9*k*a_3 - 4*a_2, 4*k^2*a_2*b_2_1 + k^2*a_11*b_2_1 + k^2*a_1*b_2_11 + k*a_11 + 4*k*a_21 - 2*a_11, k^2*a_11*b_2_1 + k*a_11 + 2*k*a_21 - 2*a_11, k^2*a_11*b_2_1 + 2*k*a_21 - 4*a_2, k^2*a_11*b_2_1 + k^2*a_1*b_2_2 + k*a_21, 2*k^2*a_11*b_2_1 + k^2*a_1*b_2_11 + k*a_111, k^2*a_11*b_2_1 + k*a_111 - 2*a_11, k^2*a_11*b_2_1 + k*a_111 - 2*a_11, k^2*a_11*b_2_1 + k*a_111 - 2*a_11, 2*k^2*a_11*b_2_1 + k^2*a_1*b_2_11 + k*a_111, k^2*a_11*b_2_1 + k*a_111 - 2*a_11, k^2*a_11*b_2_1 + k*a_111 - 2*a_11, k^2*a_11*b_2_1 + k*a_111 - 2*a_11, 2*k^2*a_11*b_2_1 + k^2*a_1*b_2_11 + k*a_111]


# Solving the system
Now, we can finally solve the system of equations! We first define all equations up to degree $n$.

In [93]:
totaleqs = trivialeqn
for i in range(n-1):
    totaleqs+=eqns(i+2)

In [94]:
a_vars=var(a_paramnames)
b_vars=var(b_paramnames)
unknowns = a_vars+b_vars

tosolve = [SR(f) == 0 for f in totaleqs]
solutions = solve(tosolve, unknowns, solution_dict=True)

In [95]:
for key,x in solutions[0].items():
    print(key)
    display(Math(latex(x.factor())))
        
        

a_1


<IPython.core.display.Math object>

a_2


<IPython.core.display.Math object>

a_11


...

<IPython.core.display.Math object>

b_5_111111


<IPython.core.display.Math object>

b_6_111111


<IPython.core.display.Math object>